In [21]:
import json
from pprint import pprint
from matplotlib import pyplot as plt
import cv2
import copy
import operator
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, ZeroPadding2D, Dropout
from keras.callbacks import TensorBoard
from keras.models import load_model
from keras.optimizers import Adam
from time import time
import os
%matplotlib qt

In [22]:
def readJson(file):
    with open(file, 'r') as fp:
        data = json.load(fp)
        
    return data

In [23]:
def read_jsons(folder):
    jsons = [json for json in os.listdir(folder) if json.endswith(".json")]
    
    return jsons

In [24]:
def data_points(jsons, json_folder_dir):
    data_points = []
    
    for i in range(0, len(jsons)):
        data_points.append([])
        with open((json_folder_dir + jsons[i])) as f:
            data = json.load(f)
        
        n_people = len(data['people'])
    
        for j in range(0, n_people):
            data_points[i].append([])
            datas = data['people'][j]['pose_keypoints_2d']
        
            for k in range(0, len(datas)):
                data_points[i][j].append(datas[k])
                
    return data_points

In [25]:
def data_classification(data):
    body_label = {0:"Nose", 1:"Neck", 2:"RShoulder", 3:"RElbow", 4:"RWrist", 5:"LShoulder", 6:"LElbow", 7:"LWrist", 8:"MidHip", 9:"RHip", 10:"RKnee", 11:"RAnkle", 12:"LHip", 13:"LKnee",14:"LAnkle", 15:"REye", 16:"LEye", 17:"REar", 18:"LEar", 19:"LBigToe", 20:"LSmallToe", 21:"LHeel", 22:"RBigToe", 23:"RSmallToe", 24:"RHeel"}
    parts = {}
    people = {}
    
    for person in range(0, len(data)):
        points = data[person]
        for i in range(0, len(points)):
            parts[body_label[i]] = points[i]
        
        people[person] = parts
        parts = {}
        
    return people

In [26]:
def organise_data(data, number):
    total = len(data)
    people_body_point = {}
    
    for i in range(0, total):
        people = len(data[i])
        frame_points = {}
        id_number = 0
        if people != 0:
            for j in range(0, people):
                points = data[i][j]
                n_points = len(points)
                body_points = []
                data_certainty = 0
                for k in range(0, n_points - 1, 3):
                    body_points.append((points[0 + k], points[1 + k]))
                    data_certainty += points[2 + k]
                    
                
                data_certainty = data_certainty / number
                
                if data_certainty >= 0.5:
                    frame_points[id_number] = body_points
                    id_number += 1
                
            organised = data_classification(frame_points)
            people_body_point[i] = organised
                
    return people_body_point

In [27]:
def reformat(data):
    
    exercise = {}
    
    for i in data:
        for j in data[i]:
            exercise[i] = data[i][j]
            
    return exercise

In [28]:
def section_separator(data, start, end):
    
    exercise_one = {}
    exercise_two = {}
    
                
    for i in range(start, end + 1):
        
        for j in data[str(i)]:
            if j == '0':
                exercise_one[i] = data[str(i)][j]
            else:
                exercise_two[i] = data[str(i)][j]
        
#         i += 1

    return exercise_one, exercise_two

In [29]:
def organiseAllPoints(data):
    new_data = {}
    
    keys = ['Nose', 'Neck', 'RShoulder', 'RElbow', 'RWrist', 'LShoulder', 'LElbow', 'LWrist', 'MidHip', 'RHip', 'RKnee', 'RAnkle', 'LHip', 'LKnee', 'LAnkle', 'REye', 'LEye', 'REar', 'LEar', 'LBigToe', 'LSmallToe', 'LHeel', 'RBigToe', 'RSmallToe', 'RHeel']
    
    # Initialisation
    for i in range(len(keys)):
        
        new_data[keys[i]] = []
       
    for keys in data:
        for Lkeys in data[keys]:
            coordinate = new_data[Lkeys]
            
            points = data[keys][Lkeys]
            
            if points[0] != 0 and points[1] != 0:
                previousPoints = copy.deepcopy(points)
                coordinate.append(points)
            else:
                coordinate.append(previousPoints)
            
#             coordinate.append(points)
            
            new_data[Lkeys] = coordinate            
    
    return new_data

In [30]:
def rescale(data):
    
    allPoints = organiseAllPoints(data)
        
    new_data = {}
    
    for keys in allPoints:
        
        new_data[keys] = []
        
    fit_points = []
    first = True
    
    for keys in allPoints:
        if first:
            fit_points = allPoints[keys]
            first = False
        else:
            fit_points + allPoints[keys]
            
    scaler = StandardScaler().fit(fit_points)
    
    for keys in allPoints:
        points = scaler.transform(allPoints[keys])
        new_data[keys] = points
        
    return new_data

In [31]:
def merge(master, slave):
    original = copy.deepcopy(master)
    for i in master:
        masterList = master[i]
        slaveList = slave[i]
        
        finalList = np.concatenate((masterList, slaveList), axis = 0)
            
        original[i] = finalList
        
    return original

In [32]:
def returnNumber(superFolder, subFolder):
    number = 25
    
    if superFolder == 'Star_Jumps':
        if subFolder == 'Starjump10':
            number = 26
        if subFolder == 'Starjump7' or 'Starjump8' or 'Starjump9':
            number = 20
        
    if superFolder == 'Squats':
        if subFolder == 'Squats_21' or subFolder == 'Squats_22':
            number = 13
            
        if subFolder == 'Squats_23':
            number = 10
            
    if superFolder == 'Biceps':
        if subFolder == 'Bicep8' or subFolder == 'Bicep9' or subFolder == 'Bicep10':
            number = 20
            
        if subFolder == 'Bicep12':
            number = 13

        if subFolder == 'Bicep13' or subFolder == 'Bicep15':
            number = 16
        
        if subFolder == 'Bicep14':
             number = 18
            
        
    if superFolder == 'Chests':
        if subFolder == 'Chest6' or subFolder == 'Chest7' or subFolder == 'Chest8':
            number = 20
            
        if subFolder == 'Chest11' or subFolder == 'Chest12':
            number = 15

        if subFolder == 'Chest13':
            number = 14
            
    if superFolder == 'Punches':
        number = 20
        
        if subFolder == 'Punches12':
            number = 14
            
        if subFolder == 'Punches7' or subFolder == 'Punches8' or subFolder == 'Punches9':
            number = 11
        
    return number

In [33]:
def read_folders(superFolder):
    subFolders = next(os.walk('Sports_Skeleton/' + superFolder + '/.'))[1]
    
    keyStart = 0
    
    total = {}
    
    first = True
    
    for i in range(len(subFolders)):
        json_folder = 'Sports_Skeleton/' + superFolder + '/' + subFolders[i]
        
        number = returnNumber(superFolder, subFolders[i]) 
            
        json_folder_slash = json_folder + '/'
        
        jsons = read_jsons(json_folder)
        
        data_points1 = data_points(jsons, json_folder_slash)
        
        part_points = organise_data(data_points1, number)
                
        reformat_points = reformat(part_points)
        
        rescale_points = rescale(reformat_points)
        
        if first:
            total = rescale_points
            first = False
        else:
            for i in rescale_points:
                total = merge(total, rescale_points)
                                
    return total

In [34]:
def dicToList(data):
    bigList = []
    first = True
    for i in data:
        if first:
            bigList = data[i]
            first = False
        else:
             bigList = np.concatenate((bigList, data[i]), axis = 0)
    
    return bigList

In [35]:
def findmaximumpeople(json_folder_dir, jsons, id0, idn):
    
    if not id0 == idn:
        midval = (id0 + idn)//2
        leftval = findmaximumpeople(json_folder_dir, jsons, id0, midval)
        rightval = findmaximumpeople(json_folder_dir, jsons, midval + 1, idn)
        
        if leftval >= rightval:
            return leftval
        else:
            return rightval
    else:
        with open((json_folder_dir + jsons[id0])) as f:
            data = json.load(f)
        
        number = 0
        
        for item in data['people']:
            data_points = item['pose_keypoints_2d']
            
            total_certainty = 0
            
            n_points = len(data_points)
            
            division = 0
            
            for k in range(0, n_points - 1, 3):
                total_certainty += data_points[2 + k]
                division += 1
                
#             total_certainty = total_certainty / 20
            total_certainty = total_certainty

        
            if (total_certainty >= 0.5):
                number += 1

                
        return number
    

In [36]:
# section_player("DS3.MOV", ds4_data, 3580, 4370)
# 210, 980
# 1430, 2210
#2550, 3330
#3580, 4370

In [37]:
json_folder = 'Sports_Skeleton/Chests/Chest4'
json_folder_slash = json_folder + '/'

jsons = read_jsons(json_folder)

number_of_people = findmaximumpeople(json_folder_slash, jsons, 0, (len(jsons) - 1))

In [38]:
number_of_people

2

In [39]:
squats0 = read_folders('Squats')

In [40]:
pprint(squats0)

{'LAnkle': array([[1.99931293, 5.74023539],
       [2.00151256, 5.76570919],
       [2.00183772, 5.76525877],
       ...,
       [7.78154141, 5.10333869],
       [7.78154141, 5.10333869],
       [7.78154141, 5.10333869]]),
 'LBigToe': array([[1.69953235, 6.34234788],
       [1.69870988, 6.31692413],
       [1.69519048, 6.34224778],
       ...,
       [6.02329372, 5.51487156],
       [6.02329372, 5.51487156],
       [6.02329372, 5.51487156]]),
 'LEar': array([[ 0.59470711, -0.76835983],
       [ 0.59512791, -0.76820969],
       [ 0.59602689, -0.76812962],
       ...,
       [ 3.89170761, -1.1413772 ],
       [ 3.89170761, -1.1413772 ],
       [ 3.89170761, -1.1413772 ]]),
 'LElbow': array([[3.20271975, 0.70059244],
       [3.0024006 , 0.54440153],
       [2.70149152, 0.49045613],
       ...,
       [5.90049262, 0.89435032],
       [5.90049262, 0.89435032],
       [5.90049262, 0.89435032]]),
 'LEye': array([[-0.5032132 , -0.95072019],
       [-0.40807452, -0.95059007],
       [-0.4062000

In [41]:
arm0 = read_folders('Biceps')

In [42]:
starJump0 = read_folders('Star_Jumps')

In [43]:
chest0 = read_folders('Chests')

In [44]:
punch0 = read_folders('Punches')

In [45]:
print(squats0)

{'Nose': array([[-1.00719576, -0.76796947],
       [-1.00622027, -0.76802452],
       [-0.90802123, -0.76832981],
       ...,
       [ 1.3722982 , -1.11843513],
       [ 1.3722982 , -1.11843513],
       [ 1.3722982 , -1.11843513]]), 'Neck': array([[-0.50351924,  0.09782935],
       [-0.50615879,  0.09738893],
       [-0.40585576,  0.07225546],
       ...,
       [ 2.62748439, -0.31806511],
       [ 2.62748439, -0.31806511],
       [ 2.62748439, -0.31806511]]), 'RShoulder': array([[-2.91459824,  0.1244392 ],
       [-3.00891446,  0.12401881],
       [-2.71439385,  0.12348832],
       ...,
       [-0.1309898 , -0.3175514 ],
       [-0.1309898 , -0.3175514 ],
       [-0.1309898 , -0.3175514 ]]), 'RElbow': array([[-5.5210807 ,  0.80584076],
       [-5.523682  ,  0.70217392],
       [-5.52440884,  0.54419133],
       ...,
       [-1.00867157,  0.8720971 ],
       [-1.00867157,  0.8720971 ],
       [-1.00867157,  0.8720971 ]]), 'RWrist': array([[-5.5210807 ,  0.80584076],
       [-5.523682  

In [46]:
ds2_data = readJson('Re-identified jsons/DS1.json')
ds3_data = readJson('Re-identified jsons/DS2.json')
ds4_data = readJson('Re-identified jsons/DS3.json')

In [47]:
squat1, arm1 = section_separator(ds2_data, 135, 800)
arm2, squat2 = section_separator(ds2_data, 1230, 1910)
squat3, arm3 = section_separator(ds2_data, 2700, 3350)
arm4, squat4 = section_separator(ds2_data, 3600, 4280)

In [48]:
squat5, arm5 = section_separator(ds3_data, 210, 930)
arm6, squat6 = section_separator(ds3_data, 1250, 1990)
squat7, arm7 = section_separator(ds3_data, 2330, 2980)
arm8, squat8 = section_separator(ds3_data, 3725, 4350)
squat9, arm9 = section_separator(ds3_data, 4585, 5325)

In [49]:
squat10, starJump1 = section_separator(ds4_data, 210, 980)
starJump2, squat11 = section_separator(ds4_data, 1430, 2210)
squat12, starJump3 = section_separator(ds4_data, 2550, 3330)
starJump4, squat13 = section_separator(ds4_data, 3580, 4370)

In [50]:
arms_rescale = {}
squats_rescale = {}
starJumps_rescale = {}

In [51]:
arms_rescale[0] = rescale(arm1)
arms_rescale[1] = rescale(arm2)
arms_rescale[2] = rescale(arm3)
arms_rescale[3] = rescale(arm4)
arms_rescale[4] = rescale(arm5)
arms_rescale[5] = rescale(arm6)
arms_rescale[6] = rescale(arm7)
arms_rescale[7] = rescale(arm8)
arms_rescale[8] = rescale(arm9)
squats_rescale[0] = rescale(squat1)
squats_rescale[1] = rescale(squat2)
squats_rescale[2] = rescale(squat3)
squats_rescale[3] = rescale(squat4)
squats_rescale[4] = rescale(squat5)
squats_rescale[5] = rescale(squat6)
squats_rescale[6] = rescale(squat7)
squats_rescale[7] = rescale(squat8)
squats_rescale[8] = rescale(squat9)
squats_rescale[9] = rescale(squat10)
squats_rescale[10] = rescale(squat11)
squats_rescale[11] = rescale(squat12)
squats_rescale[12] = rescale(squat13)
starJumps_rescale[0] = rescale(starJump1)
starJumps_rescale[1] = rescale(starJump2)
starJumps_rescale[2] = rescale(starJump3)
starJumps_rescale[3] = rescale(starJump4)

In [52]:
for i in range(0, 9):
    arm0 = merge(arm0, arms_rescale[i])

In [53]:
for i in range(0, 13):
    squats0 = merge(squats0, squats_rescale[i])

In [54]:
for i in range(0, 4):
    starJump0 = merge(starJump0, starJumps_rescale[i])

In [55]:
arm_size = len(arm0[list(arm0.keys())[0]])

In [56]:
squat_size = len(squats0[list(squats0.keys())[0]])

In [57]:
starJump_size = len(starJump0[list(starJump0.keys())[0]])

In [58]:
chest_size = len(chest0[list(chest0.keys())[0]])

In [59]:
punch_size = len(punch0[list(punch0.keys())[0]])

In [60]:
arms = dicToList(arm0)

In [61]:
squats = dicToList(squats0)

In [62]:
starJumps = dicToList(starJump0)

In [63]:
chests = dicToList(chest0)

In [64]:
punches = dicToList(punch0)

In [65]:
print(arms.shape)
print(squats.shape)
print(starJumps.shape)
print(chests.shape)
print(punches.shape)

(24465250, 2)
(21830650, 2)
(27334875, 2)
(18511625, 2)
(24457400, 2)


In [66]:
arms_shape = arms.reshape(arm_size, 50)
print(arms_shape.shape)

(978610, 50)


In [67]:
squats_shape = squats.reshape(squat_size, 50)
print(squats_shape.shape)

(873226, 50)


In [68]:
starJumps_shape = starJumps.reshape(starJump_size, 50)
print(starJumps_shape.shape)

(1093395, 50)


In [69]:
chests_shape = chests.reshape(chest_size, 50)
print(chests_shape.shape)

(740465, 50)


In [70]:
punches_shape = punches.reshape(punch_size,50)
print(punches_shape.shape)

(978296, 50)


In [71]:
if len(arms_shape) % 30 != 0:
    arms_shape = arms_shape[:-(len(arms_shape) % 30), :]

if len(squats_shape) % 30 != 0:
    squats_shape = squats_shape[:-(len(squats_shape) % 30), :]
    
if len(starJumps_shape) % 30 != 0:
    starJumps_shape = starJumps_shape[:-(len(starJumps_shape) % 30), :]

if len(chests_shape) % 30 != 0:
    chests_shape = chests_shape[:-(len(chests_shape) % 30), :]

if len(punches_shape) % 30 != 0:
    punches_shape = punches_shape[:-(len(punches_shape) % 30), :]

print(arms_shape.shape)
print(squats_shape.shape)
print(starJumps_shape.shape)
print(chests_shape.shape)
print(punches_shape.shape)

(978600, 50)
(873210, 50)
(1093380, 50)
(740460, 50)
(978270, 50)


In [72]:
final_arm = arms_shape.reshape(int(len(arms_shape) / 30), 30, 50, 1)
print(final_arm.shape)

(32620, 30, 50, 1)


In [73]:
final_squats = squats_shape.reshape(int(len(squats_shape)/30), 30, 50, 1)
print(final_squats.shape)

(29107, 30, 50, 1)


In [74]:
final_starJumps = starJumps_shape.reshape(int(len(starJumps_shape)/30), 30, 50, 1)
print(final_starJumps.shape)

(36446, 30, 50, 1)


In [75]:
final_chests = chests_shape.reshape(int(len(chests_shape)/30), 30, 50, 1)
print(final_chests.shape) 

(24682, 30, 50, 1)


In [76]:
final_punches = punches_shape.reshape(int(len(punches_shape)/30), 30, 50, 1)
print(final_punches.shape)

(32609, 30, 50, 1)


In [77]:
label = np.array([])

for i in range(int(len(arms_shape) / 30)):
    label = np.append(label, 0)

for i in range(int(len(squats_shape)/30)):
    label = np.append(label, 1)
    
for i in range(int(len(starJumps_shape)/30)):
    label = np.append(label, 2)
    
for i in range(int(len(chests_shape)/30)):
    label = np.append(label, 3)

for i in range(int(len(punches_shape)/30)):
    label = np.append(label, 4)
    
print(label.shape)
    

(155464,)


In [78]:
x_data = np.concatenate((final_arm, final_squats, final_starJumps, final_chests, final_punches))

In [79]:
x_data.shape

(155464, 30, 50, 1)

In [80]:
label = label.reshape((int(len(arms_shape) / 30)) + (int(len(squats_shape) / 30)) + (int(len(starJumps_shape) / 30)) + (int(len(chests_shape)/30)) + (int(len(punches_shape)/30)), 1)

In [81]:
label.shape

(155464, 1)

In [82]:
X_train, X_test, y_train, y_test = train_test_split(x_data, label, test_size = 0.3, random_state = 101 )

In [83]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [84]:
X_train.shape

(108824, 30, 50, 1)

In [85]:
model = Sequential()
model.add(Conv2D(64, kernel_size = 5, activation = 'relu', input_shape = (30, 50, 1)))

model.add(ZeroPadding2D((1,1)))
model.add(Dropout(0.1))
model.add(Conv2D(32, kernel_size = 4, activation = 'relu'))

model.add(ZeroPadding2D((1,1)))
model.add(Dropout(0.1))
model.add(Conv2D(16, kernel_size = 3, activation = 'relu'))

model.add(ZeroPadding2D((1,1)))
model.add(Dropout(0.1))
model.add(MaxPooling2D(pool_size = (8,8), strides = (3,3)))
model.add(Flatten())
model.add(Dense(5, activation='softmax'))

tensorboard = TensorBoard(log_dir='log/{}'.format(time()))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [86]:
model.compile(optimizer = Adam(lr = 0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [87]:
model.fit(X_train, y_train, validation_data =(X_test, y_test), epochs=100, callbacks=[tensorboard], verbose=False, shuffle=True)

Instructions for updating:
Use tf.cast instead.


In [88]:
model.save('model/Exercise_Classification_Shuffle_Fixed_Data_Test.h5')

In [89]:
del model